In [1]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes"
# folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
folders = ['webd-dependency', 'combined-fasttext', 'essays-dependency',
          'original', 'essays-fasttext', 'ibm-fasttext', 'webd-fasttext']
objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            if subdir.endswith('100'):
                print(path)
                indexref = pt.IndexRef.of(path)
                index = pt.IndexFactory.of(indexref)
                bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
                tuned_bm25, parameters = pt.KFoldGridSearch(
                    bm25,
                    {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                    "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                    "bm25.k_3": [2, 5, 8, 10]
                    }},
                    [topics20, topics21],
                    [qrels20, qrels21],
                    objective
                )
                
                print(pt.Experiment([tuned_bm25], topics, qrels, names=[folders[i] + '_tuned_bm25_'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([tuned_bm25], topics20, qrels, names=[folders[i] + '_tuned_bm25_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([tuned_bm25], topics21, qrels, names=[folders[i] + '_tuned_bm25_21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

                print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm25__unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm25__unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_bm25__unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                tuned_bm25.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_bm25.csv")
                i += 1
            

/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_webd-dependency-100/data.properties
Fold 1
Best ndcg_cut_5 is 0.374316
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=1.2', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.263423
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=1.6', 'BR(BM25) bm25.k_3=2']


/tmp/ipykernel_3684173/4231427746.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25], topics, qrels, names=[folders[i] + '_tuned_bm25_'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-dependency\_tuned\_bm25\_ &    0.314561 &     0.250217 &  0.423577 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3684173/4231427746.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25], topics20, qrels, names=[folders[i] + '_tuned_bm25_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-dependency\_tuned\_bm25\_20 &     0.25835 &     0.224005 &  0.473815 \\
\bottomrule
\end{tabular}



...

\begin{tabular}{llrrr}
\toprule
{} &                       name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_bm25\_ &    0.309314 &     0.235246 &  0.407214 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3684173/4231427746.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25], topics20, qrels, names=[folders[i] + '_tuned_bm25_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_bm25\_20 &    0.241371 &     0.187997 &  0.440268 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3684173/4231427746.py:34: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25], topics21, qrels, names=[folders[i] + '_tuned_bm25_21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_3684173/4231427746.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm25__unjudged'], eval_metrics=["ndcg_

\begin{tabular}{llrrr}
\toprule
{} &                         name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_bm25\_21 &    0.375898 &     0.281551 &  0.374822 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_bm25\_\_unjudged &    0.590416 &     0.488071 &  0.407214 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_bm25\_\_unjudged20 &    0.665406 &     0.535553 &  0.440268 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                  name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  webd-fasttext\_tuned\_bm25\_\_unjudged21 &    0.577869 &     0.507808 &  0.374822 \\
\bottomrule
\end{tabular}



In [5]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']

objective="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file

            print(path)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
            tuned_bm25, parameters = pt.KFoldGridSearch(
                bm25,
                {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                "bm25.k_3": [2, 5, 8, 10]
                }},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )

            print(pt.Experiment([tuned_bm25], topics, qrels, names=[folders[i] + '_tuned_bm25_'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_bm25], topics20, qrels, names=[folders[i] + '_tuned_bm25_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_bm25], topics21, qrels, names=[folders[i] + '_tuned_bm25_21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

            print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm25__unjudged'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm25__unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] + '_tuned_bm25__unjudged21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())

            i += 1
            

/workspace/src/grid_search_right_qrels/super_folder//data.properties
Fold 1
Best ndcg_cut_5 is 0.460400
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=3.4', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.278232
Best setting is ['BR(BM25) c=0.75', 'BR(BM25) bm25.k_1=0.6', 'BR(BM25) bm25.k_3=2']


/tmp/ipykernel_2391368/552348811.py:31: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25], topics, qrels, names=[folders[i] + '_tuned_bm25_'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_bm25\_ &    0.329196 &     0.304758 &  0.622564 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391368/552348811.py:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25], topics20, qrels, names=[folders[i] + '_tuned_bm25_20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                            name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_bm25\_20 &    0.257784 &     0.261112 &  0.621228 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391368/552348811.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25], topics21, qrels, names=[folders[i] + '_tuned_bm25_21'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391368/552348811.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos)]], topics, qrels, names=[folders[i] + '_tuned_bm25__unjudged'], eval_metrics=["ndcg_cu

\begin{tabular}{llrrr}
\toprule
{} &                            name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_bm25\_21 &    0.399179 &     0.347531 &  0.623873 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                   name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_bm25\_\_unjudged &    0.556162 &     0.601818 &  0.622564 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_bm25\_\_unjudged20 &    0.606965 &     0.637791 &  0.621228 \\
\bottomrule
\end{tabular}

\begin{tabular}{llrrr}
\toprule
{} &                                     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  marco-pretrained\_tuned\_bm25\_\_unjudged21 &    0.529812 &     0.609965 &  0.623873 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_2391368/552348811.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos20)]], topics20, qrels, names=[folders[i] + '_tuned_bm25__unjudged20'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
/tmp/ipykernel_2391368/552348811.py:37: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25[tuned_bm25['docno'].isin(docnos21)]], topics21, qrels, names=[folders[i] +